### Install required package to encrypt and decrypt

In [11]:
!pip3 install cryptography

In [12]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes
import os
import base64

In [13]:
# Generate a key from a password
def generate_key(password, salt):
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=100000,
        backend=default_backend()
    )
    key = kdf.derive(password.encode())
    return key

In [14]:
# Encrypt the message
def encrypt_message(key, message):
    iv = os.urandom(16)
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    
    # Padding the message to match block size (16 bytes)
    padder = padding.PKCS7(algorithms.AES.block_size).padder()
    padded_message = padder.update(message.encode()) + padder.finalize()

    encrypted_message = encryptor.update(padded_message) + encryptor.finalize()

    # Return the IV and encrypted message, both base64-encoded
    return base64.b64encode(iv + encrypted_message).decode()

In [15]:
# Decrypt the message
def decrypt_message(key, encrypted_message):
    encrypted_message = base64.b64decode(encrypted_message.encode())
    iv = encrypted_message[:16]
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    decryptor = cipher.decryptor()

    decrypted_message = decryptor.update(encrypted_message[16:]) + decryptor.finalize()

    # Unpad the message to original form
    unpadder = padding.PKCS7(algorithms.AES.block_size).unpadder()
    unpadded_message = unpadder.update(decrypted_message) + unpadder.finalize()

    return unpadded_message.decode()

In [16]:
password = "rmit#%hackathon#group$binhduong"

salt = os.urandom(16)

message = "Welcome to RMIT Hackathon 2024 from group Binh Duong"

key = generate_key(password, salt)

assert key is not None

In [17]:
encrypted_message = encrypt_message(key, message)
print(f"Encrypted: {encrypted_message}")

decrypted_message = decrypt_message(key, encrypted_message)
print(f"Decrypted: {decrypted_message}")

Encrypted: /w9OyHzGlA4g0fcI475T6OW+yfrXM7i3desqtaMONW7F3KLoiJMfM9PbEbDdqLLRzgRJDGijAiXg4uBJYY0evaFu08CfnkUuIQOCd9BslHg=
Decrypted: Welcome to RMIT Hackathon 2024 from group Binh Duong


In [20]:
import requests
import json



api_key = 'voy-5uh12i196p8rqqgo3pbdmoaj1teit9clgk9ujkk0n91o'

url = 'http://3.27.163.79:8000/v1/chat/completions'

headers = {
    'Authorization': f'Bearer {api_key}',
    'Content-Type': 'application/json',
    'Accept': 'application/json'
}



def send_chat_message(message):
    decrypted_message = decrypt_message(key, message)
    print(f'Decrypted message : {decrypted_message}')
    payload = {
        'messages': [
            {
                "role": "system",
                "content": "You are a helpful assistant who helps users solve their questions."
            },
            {
                "role": "user",
                "content": decrypted_message
            }
        ]
    }

    try:
        response = requests.post(url, json=payload, headers=headers)
        response.raise_for_status()
        if response.status_code == 200:
            try:
                json_response = response.json()
                print(f'Response Status : {response.status_code}')
                print('Role : ',end='')
                print(json_response['choices'][0]['message']['role'])
                print('Message : ',end='')

                encrRes = encrypt_message(key,json_response['choices'][0]['message']['content'])
                print(encrRes)
                print('Decrypted')
                print(decrypt_message(key, encrRes))

            except json.JSONDecodeError as e:
                print(f'Response Status : {response.status_code}')
                print(e)
        else:
            print(f'Response Status : {response.status_code}')
    except requests.exceptions.RequestException as e:
        print(e)

message = 'Hi, how are you!'
encrypted_message = encrypt_message(key, message)

print(f'Encrypted message : {encrypted_message}')
send_chat_message(encrypted_message)

Encrypted message : Rs7nsgCE2cMfBAtTO09XxQmnEV3F6EyphUZntGwcTv90kF733sJve+UvL5ALTT6N
Decrypted message : Hi, how are you!
Response Status : 200
Role : assistant
Message : DfhInoCsdTm/4bHm52CQnmpvqY0HD1CS/FvecPwG0gzUXCvBCvpGuyVLpQ+p28SvR7b9bASh6AMPyLgxAoDsnP9UA3JwjuviNRSlewc+oL3aulPJUHviGcDaSbagoDsRkSF96VawljDvY7mTe+WhCHE7tEdUMbKnSloKx9nl/tXwHATkpdnDzYpMFKZC8D6iwbyFpy/AXScEWWl2wJMtxCmzMv8duEByj4G990P3haGL+tvYE/AIkBV3INxY00EuhWDYBrB0BkE8vZMGVhEoppIetDKCnVzhwe8Y/vAdN/4OHJPGXKjn2DlWi94iSYDA6KmoZ/xPTGGa9fLw5RHl6qn1sxx2f+/blo0jXwhQTkda6L9CussMxsCd8v8s1QnG/Hp0A0VhxZFNm9fPPuBKB6dsgC/pSDpmqRXjRaDCOdtQOgdpSYfc4WBx+JXTfB/y
Decrypted
I'm an AI assistant that's been working on your site for years. I'm here to help you with your questions and help you understand how you feel about your site. I'll do my best to provide accurate and helpful responses while ensuring that I don't make any mistakes or promote any harmful or unethical content.

